In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import Tensor

import gym
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from itertools import count
from typing import Union, Tuple, Callable

## Replay Buffer

In [ ]:
class ReplayBuffer(object):
    def __init__(self, size: int = 10000):
        """
        Constructor
        
        Parameters
        ----------
        size
            Maximum number of transitions store in the buffer.
            If the buffer overflows, older states are dropped.
        
        """
        self.size    = size
        self.length  = 0
        self.idx     = -1
        
        # define buffers
        self.states        = None
        self.states_next   = None
        self.actions       = None
        self.rewards       = None
        self.done          = None
        
    def store(self, 
              s: Union[torch.Tensor, np.ndarray], 
              a: int, 
              r: float, 
              s_next: Union[torch.Tensor, np.ndarray],
              done: bool):
        
        """
        Stores one sample of experience
        
        Parameters
        ----------
        s
            Tensor encoding the current state.
        a
            Current action.
        r
            Current reward.
        s_next
            Tensor encoding the next state.
        done
            Done signal.
        """
        
        # initialize buffers
        if self.states is None:
            self.states      = torch.zeros([self.size] + list(s.shape))
            self.states_next = torch.zeros_like(self.states)
            self.actions     = torch.zeros((self.size, ))
            self.rewards     = torch.zeros((self.size, ))
            self.done        = torch.zeros((self.size, ))
        
        self.idx = (self.idx + 1) % self.size
        self.length = min(self.length + 1, self.size)
        
        if type(s) != Tensor:
            s = Tensor(s)
            
        if type(s_next) != Tensor:
            s_next = Tensor(s_next)
        
        # store the experience
        self.states[self.idx]       = s
        self.states_next[self.idx]  = s_next
        self.actions[self.idx]      = a
        self.rewards[self.idx]      = r
        self.done[self.idx ]        = int(done)
        
    def sample(self, batch_size: int = 128) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor]:
        """
        Sample a batch of experience
        
        Parameters
        ----------
        batch_size
            Number of experience to sample
            
        Returns
        -------
        Tuple of tensor consisting of a batch of states, actions, rewards, next states, done
        """
        
        assert self.length >= batch_size, "Can not sample from the buffer yet"
        indices = np.random.choice(a=np.arange(self.length), size=batch_size, replace=False)
        
        s      = self.states[indices]
        s_next = self.states_next[indices]
        a      = self.actions[indices]
        r      = self.rewards[indices]
        done   = self.done[indices]
        return s, a, r, s_next, done

## Network achitecture

In [ ]:
class DQN_RAM(nn.Module):
    def __init__(self, in_features: int, num_actions: int):
        """
        Initialize a deep Q-learning network for testing algorithm
        
        Parameters
        ----------
        in_features
            number of features of input.
        num_actions
            number of action-value to output, one-to-one correspondence to action in game.
        """
        super(DQN_RAM, self).__init__()
        self.in_features = in_features
        self.num_actions = num_actions
        
        # define architecture
        self.fc1 = nn.Linear(in_features, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, num_actions)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

## Epsilon scheduler

In [ ]:
def eps_generator(max_eps: float=1.0, min_eps: float=0.1, max_iter: int = 10000):
    crt_iter = -1
    
    while True:
        crt_iter += 1
        frac = min(crt_iter/max_iter, 1)
        eps = (1 - frac) * max_eps + frac * min_eps
        yield eps

## Epsilon greedy policy

In [ ]:
def select_epilson_greedy_action(Q: nn.Module, s: Tensor, eps: float):
    rand = np.random.rand()
    
    # with prob eps select a random action
    if rand < eps:
        return np.random.choice(np.arange(Q.num_actions))
    
    # select best action
    with torch.no_grad():
        output = Q(s).argmax(dim=1).item()
    
    return output

## DQN target

In [ ]:
def dqn_target(
    Q: nn.Module,
    target_Q: nn.Module,
    r_batch: Tensor,
    s_next_batch: Tensor,
    done_batch: Tensor,
    gamma: float) -> Tensor:
    """
    Computes DQN target
    
    Parameters:
    -----------
    Q
        Behavior Q network.
    target_Q
        Target Q network.
    r_batch
        Batch of rewards.
    s_next_bacth
        Batch of next states.
    done_batch
        Batcho of done flag (1 means the episoded finished).
    gamma
        Discount factor.
    
    Returns
    -------
    Batch of DQN targets
    """
    # cmpute next Q value based on which action gives max Q values
    # detach variable from the current graph since we don't want gradients for next Q to propagated
    next_max_Q    = torch.max(target_Q(s_next_batch).detach(), dim=1)[0]
    next_Q_values = (1 - done_batch) * next_max_Q
    return r_batch + (gamma * next_Q_values)

## DDQN target

In [ ]:
def ddqn_target(
    Q: nn.Module,
    target_Q: nn.Module,
    r_batch: Tensor,
    s_next_batch: Tensor,
    done_batch: Tensor,
    gamma: float) -> Tensor:
    """
    Computes DQN target
    
    Parameters:
    -----------
    Q
        Behavior Q network.
    target_Q
        Target Q network.
    r_batch
        Batch of rewards.
    s_next_bacth
        Batch of next states.
    done_batch
        Batcho of done flag (1 means the episoded finished).
    gamma
        Discount factor.
    
    Returns
    -------
    Batch of DQN targets
    """
    # cmpute next Q value based on which action gives max Q values
    # detach variable from the current graph since we don't want gradients for next Q to propagated
    next_max_Q = target_Q(s_next_batch).gather(1, Q(s_next_batch).argmax(dim=1, keepdim=True))
    next_max_Q = next_max_Q.view(-1).detach()
    next_Q_values = (1 - done_batch) * next_max_Q

    # Compute the target of the current Q values
    return r_batch + (gamma * next_Q_values)

##  Learning Alogrithm

In [ ]:
def learing(
    env: gym.Env,
    targert_function: Callable,
    batch_size: int = 128,
    gamma: float = 0.99,
    replay_buffer_size=10000,
    num_episodes: int = 100000,
    learning_starts: int = 1000,
    learning_freq: int = 4,
    target_update_freq: int = 100,
    log_every: int = 100):

    """
    DQN Learning
    
    Parameters
    ----------
    env
        gym environment to train on.
    target_function
        Function that computes the Q network target. For DQN - dqn_target, for DDQN - ddqn_target.
    batch_size:
        How many transitions to sample each time experience is replayed.
    gamma
        Discount Factor
    replay_buffer_size
        Replay buffer size.
    num_episodes
        number of episodes to run
    learning_starts: int
        After how many environment steps to start replaying experiences
    learning_freq: int
        How many steps of environment to take between every experience replay
    target_update_freq: int
        How many experience replay rounds (not steps!) to perform between
        each update to the target Q network
    log_every:
        Logging interval
    """
    # This means we are running on low-dimensional observations (e.g. RAM)
    input_arg = env.observation_space.shape[0]
    num_actions = env.action_space.n

    # define device 
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Initialize target q function and q function
    Q = DQN_RAM(input_arg, num_actions).to(device)
    target_Q = DQN_RAM(input_arg, num_actions).to(device)
      
    # Construct Q network optimizer function
    optimizer = optim.Adam(Q.parameters(), lr=1e-3)
    
    # define criterion
    criterion = nn.MSELoss()

    # Construct the replay buffer
    replay_buffer = ReplayBuffer()
    
    # define epsilon scheduler
    eps_scheduler = iter(eps_generator())
    
    # define statistics buffer, total number of steps and total number of updates performed
    all_episode_rewards = []
    total_steps = 0
    num_param_updates = 0
    
    for episode in range(1, num_episodes + 1):
        # reset environment
        s = env.reset()
        episode_reward = 0
        
        for _ in count():
            # increse total number of steps
            total_steps += 1
            
            # Choose random action if not yet start learning
            if total_steps > learning_starts:
                eps = next(eps_scheduler)
                s = torch.tensor(s).view(1, -1).float().to(device)
                a = select_epilson_greedy_action(Q, s, eps)
            else:
                a = np.random.choice(np.arange(num_actions))

            # advance one step
            s_next, r, done, _ = env.step(a)
            
            # update episode rewards
            episode_reward += r

            # store other info in replay memory
            replay_buffer.store(s, a, r, s_next, done)

            # Resets the environment when reaching an episode boundary.
            if done:
                break

            # update state
            s = s_next

            # perform experience replay and train the network.
            if (total_steps > learning_starts and total_steps % learning_freq == 0):
                for _ in range(learning_freq):
                    # sample experinence from the replay buffer
                    s_batch, a_batch, r_batch, s_next_batch, done_batch = replay_buffer.sample(batch_size)

                    # send everything to device
                    s_batch      = s_batch.float().to(device)
                    a_batch      = a_batch.long().to(device)
                    r_batch      = r_batch.float().to(device)
                    s_next_batch = s_next_batch.float().to(device)
                    done_batch   = done_batch.long().to(device)

                    # comput the q values according to the states and actions
                    Q_values = Q(s_batch).gather(1, a_batch.unsqueeze(1)).view(-1)

                    # Compute the target of the current Q values
                    target_Q_values = targert_function(Q, target_Q, r_batch, s_next_batch, done_batch, gamma)

                    # compute loss
                    loss = criterion(target_Q_values, Q_values)

                    # Clear previous gradients before backward pass
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    # increase number of updates
                    num_param_updates += 1

                    # Periodically update the target network by Q network to target Q network
                    if num_param_updates % target_update_freq == 0:
                        target_Q.load_state_dict(Q.state_dict())

        # append total reward culumated
        all_episode_rewards.append(episode_reward)
        
        # log average reward over the last 100 episodes
        if episode % log_every == 0 and total_steps > learning_starts:
            mean_episode_reward = np.mean(all_episode_rewards[-100:])
            print("Episode: %d, Mean reward: %.2f, Eps: %.2f" % (episode, mean_episode_reward, eps))


## DQN Learning

In [ ]:
# initialize gym env
env = gym.make("CartPole-v1")

# DQN learning
learing(
    env=env,                          # gym environmnet
    targert_function=dqn_target,      # dqn target construction
    batch_size=128,                   # q-network update batch size
    gamma=0.99,                       # discount factor
    replay_buffer_size=10000,         # size of the replay buffer
    num_episodes=1000,                # number of episodes to run
    learning_starts=1000,             # number of initial random actions (exploration)
    learning_freq=4,                  # frequency of the update
    target_update_freq=100,           # number of gradient steps after which the target network is updated
    log_every=100                     # logging interval. returns the mean reward per episode.
)

## DDQN Learning

In [ ]:
# initialize gym env
env = gym.make("CartPole-v1")

# DQN learning
learing(
    env=env,                          # gym environmnet
    targert_function=ddqn_target,     # dqn target construction
    batch_size=128,                   # q-network update batch size
    gamma=0.99,                       # discount factor
    replay_buffer_size=10000,         # size of the replay buffer
    num_episodes=1000,                # number of episodes to run
    learning_starts=1000,             # number of initial random actions  (exploration)
    learning_freq=4,                  # frequency of the update
    target_update_freq=100,           # number of gradient steps after which the target network is updated
    log_every=100                     # logging interval. returns the mean reward per episode.
)